In [1]:
#import sksurv
from sksurv.datasets import load_gbsg2
from survONS import surv_ons

x, y = load_gbsg2()

In [2]:
def cancer_levels(val):
    if val == "I":
        return 1
    elif val == "II":
        return 2
    elif val == "III":
        return 3
    else:
        return 0

In [3]:
x["horTh"] = x["horTh"].cat.codes
x["menostat"] = x["menostat"].apply(lambda val: 0 if (val == "Pre") else 1)
x["tgrade"] = x["tgrade"].apply(lambda val: cancer_levels(val))
x.head()

,age,estrec,horTh,menostat,pnodes,progrec,tgrade,tsize
0,70.0,66.0,0,1,3.0,48.0,2,21.0
1,56.0,77.0,1,1,7.0,61.0,2,12.0
2,58.0,271.0,1,1,9.0,52.0,2,35.0
3,59.0,29.0,1,1,4.0,60.0,2,17.0
4,73.0,65.0,0,1,1.0,26.0,2,35.0


In [4]:
import pandas as pd

df = pd.DataFrame(y)
df.head(10)

,cens,time
0,True,1814.0
1,True,2018.0
2,True,712.0
3,True,1807.0
4,True,772.0
5,True,448.0
6,False,2172.0
7,False,2161.0
8,True,471.0
9,False,2014.0


In [6]:
import numpy as np

# Simulación sin dataset
N2 = 1000
n_it2 = 500
max_T = 1000
d2 = 4
eta2 = np.zeros((3,))
sigma2 = np.eye(3,3)
gamma2 = np.arange(np.log(1/n_it2), np.log(50), 1.2)
gamma2 = np.exp(gamma2)
K2 = len(gamma2)

M2 = 1
beta_real2 = np.zeros((d2, M2))
grid_bound2 = np.zeros((M2, 1))
beta_surv2 = np.zeros((n_it2,d2,M2))
like_surv2 = np.zeros((n_it2, M2))
gamma_t2 = np.zeros((n_it2, M2))

for i in range(M2):
    beta_real2[: , i] = np.random.normal(loc=0, scale=1, size=d2)
    D2 = 1.1 * np.sqrt(np.dot(beta_real2[:, i].T, beta_real2[:, i]))
    epsilon = 1/(gamma2*D2) ** 2
    
    X2 = np.random.multivariate_normal(eta2, sigma2, N2)
    X2_1 = np.column_stack((np.ones(N2), X2))
    
    arrival_time = np.random.uniform(0, max_T, N2)
    random_time = lambda x: x + (max_T - x) * np.random.rand()
    time_indiv = np.array([random_time(xi) for xi in arrival_time])
    censor_indiv = np.array([random_time(xi) for xi in arrival_time])
    # time_indiv  = arrival_time + np.array([np.random.exponential(
    #     1 / np.exp(np.dot(beta_real2[:, i].T, X2_1[j, ])))
    #      for j in range(N2)])
    # censor_indiv = arrival_time + np.array([np.random.exponential(
    #     1 / np.exp(np.dot(beta_real2[:, i].T, X2_1[j, ])))
    #      for j in range(N2)])
    u2 = np.array([min(time_indiv[i], censor_indiv[i]) for i in range(N2)])
    delta = time_indiv < censor_indiv
    
    R2 = [[] for _ in range(n_it2)]

    for t in range(0, n_it2):
        R2[t].append(0)
    for i in range(1, N2):
        t1 = max(1,int(np.floor(arrival_time[i])-1)) # tiempo en el que i entra al estudio
        t2 = min(n_it2,int(np.floor(u2[i])+1)) # tiempo en el que i sale del estudio
        for t in range(t1, t2):
            R2[t].append(i)
            
    # print("time_indiv", time_indiv)
    # print("censor_indiv", censor_indiv)
    # print("arrival_time", arrival_time)
    # print("delta", delta) # en R es un arreglo de booleanos
    # print("u2", u2)
    # print("X2_1", type(X2_1), len(X2_1), X2_1.shape)
    # print("D2", D2)
    # print("gamma2 ",type(gamma2), len(gamma2), gamma2.shape)
    # print("n_it2", n_it2)
    # print("epsilon",type(epsilon), len(epsilon), epsilon.shape)
    # print("R2 ",type(R2), len(R2))
    # print(R2)

    
    surv = surv_ons(arrival_time, u2, delta, X2_1, D2, gamma2, n_it2, epsilon, R2)
    # print(surv)

--------------------------------------------------------------
iteracion 1
beta (instgrad) [[0.]
 [0.]
 [0.]
 [0.]]
HOLA soy el for
lik 0.0
hess [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
grad [[0.]
 [0.]
 [0.]
 [0.]]
HOLA soy el for
lik -0.00020633500010425098
hess [[0.00023382 0.00023382 0.00023382 0.00023382]
 [0.00023382 0.00023382 0.00023382 0.00023382]
 [0.00023382 0.00023382 0.00023382 0.00023382]
 [0.00023382 0.00023382 0.00023382 0.00023382]]
grad [[ 2.06335000e-04]
 [ 8.31581462e-06]
 [-7.15746714e-05]
 [ 2.18815572e-05]]
HOLA soy el for
lik -0.00020633500010425098
hess [[0.00023382 0.00023382 0.00023382 0.00023382]
 [0.00023382 0.00023382 0.00023382 0.00023382]
 [0.00023382 0.00023382 0.00023382 0.00023382]
 [0.00023382 0.00023382 0.00023382 0.00023382]]
grad [[ 2.06335000e-04]
 [ 8.31581462e-06]
 [-7.15746714e-05]
 [ 2.18815572e-05]]
HOLA soy el for
lik -0.0004444213629651843
hess [[0.001115 0.001115 0.001115 0.001115]
 [0.001115 0.001115 0.001115 0.001115

KeyboardInterrupt: 

In [14]:
N = x.shape[0]  #Número de individuosid
n_it = 2
max_t = df["time"].max()
d = x.shape[1]

eta = np.zeros((3,1))
sigma = np.eye(3,3)

gamma = np.arange(np.log(1/n_it), np.log(50), 1.2)
gamma = np.exp(gamma)

K = len(gamma)
M = 5

beta_real = np.zeros((d, M))
grid_bound = np.zeros((M, 1))

beta_surv = np.zeros((n_it,d,M))

like_surv = np.zeros((n_it, M))

gamma_t = np.zeros((n_it, M))

In [15]:
# for i in range(M):
#     beta_real[: , i] = np.random.normal(loc=0, scale=1, size=d)
#     D = 1.1 * np.cross(beta_real[:, i])
#     epsilon = 1/(gamma*D) ** 2
    
#     X = np.random.multivariate_normal(eta, sigma, N)

X = x.to_numpy()
t0 = np.zeros(N)
u = y["time"]
delta = y["cens"]
# sqrt(crossprod(beta_real[,m]))[1,]
D = 30
n = n_it
epsilon = 1/(gamma*D) ** 2
R = [[] for _ in range(n_it)]

for t in range(n_it):
    # R[t].append(0)
    for i in range(N):
        t1 = 0
        t2 = min(max_t * t/n_it, u[i]) # asumiendo tiempos enteros
        R[t].append(0 if u[i] <= max_t * t/n_it else 1)

In [ ]:
survons = surv_ons(t0, u, delta, X, D, gamma, n, epsilon, R)
#print(survons[5])

In [2]:
def cancer_levels(val):
    if val == "I":
        return 1
    elif val == "II":
        return 2
    elif val == "III":
        return 3
    else:
        return 0

In [4]:
import pandas as pd

df = pd.DataFrame(y)
df.head(10)

,cens,time
0,True,1814.0
1,True,2018.0
2,True,712.0
3,True,1807.0
4,True,772.0
5,True,448.0
6,False,2172.0
7,False,2161.0
8,True,471.0
9,False,2014.0


In [14]:
N = x.shape[0]  #Número de individuosid
n_it = 2
max_t = df["time"].max()
d = x.shape[1]

eta = np.zeros((3,1))
sigma = np.eye(3,3)

gamma = np.arange(np.log(1/n_it), np.log(50), 1.2)
gamma = np.exp(gamma)

K = len(gamma)
M = 5

beta_real = np.zeros((d, M))
grid_bound = np.zeros((M, 1))

beta_surv = np.zeros((n_it,d,M))

like_surv = np.zeros((n_it, M))

gamma_t = np.zeros((n_it, M))

In [ ]:
survons = surv_ons(t0, u, delta, X, D, gamma, n, epsilon, R)
#print(survons[5])